<a href="https://colab.research.google.com/github/Gahan25/Maersk-Case-Study/blob/main/SimulatingacontainerterminalusingSimPy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install simpy

In [3]:
import simpy
import random

In [4]:
AVERAGE_ARRIVAL_INTERVAL = 300  # 5 hours
NUM_TRUCKS = 3


In [5]:
class Vessel:
    """Class representing a vessel in the container terminal simulation."""

    def __init__(self, env, name):
        """Initialize a vessel with the given name and environment."""
        self.env = env
        self.name = name
        self.containers = 150

    def berth_vessel(self, berth):
        """Process for berthing the vessel at a berth."""
        req = berth.request()
        yield req
        print(f"{self.env.now}: {self.name} berthed")
        yield self.env.timeout(0.1)
        berth.release(req)

    def unload_containers(self, crane, truck):
        """Process for unloading containers from the vessel."""
        print(f"{self.env.now}: Unloading started for {self.name}\n")
        while self.containers > 0:
            crane_req = crane.request()
            yield crane_req
            yield self.env.timeout(3)
            self.containers -= 1
            print(f"{self.env.now}: {150 - self.containers} container unloaded from {self.name}")
            crane.release(crane_req)

            truck_req = truck.request()
            yield truck_req
            print(f"{self.env.now}: Container assigned to truck")
            yield self.env.timeout(6)
            print(f"{self.env.now}: Truck returned")
            truck.release(truck_req)

In [6]:
def vessel_generator(env, berth, crane, trucks):
    """Generator function for creating vessels and assigning them to berths and cranes."""
    vessel_count = 0
    while True:
        vessel_count += 1
        vessel_name = f"Vessel-{vessel_count}"
        vessel = Vessel(env, vessel_name)

        env.process(vessel.berth_vessel(berth))
        env.process(vessel.unload_containers(crane, random.choice(trucks)))

        arrival_interval = random.expovariate(1 / AVERAGE_ARRIVAL_INTERVAL)
        yield env.timeout(arrival_interval)

In [7]:
def main():
    SIMULATION_TIME = input("Provide the simulation time in minutes: ")
    if not SIMULATION_TIME.isdigit():
        print("The input could not be parsed. Please provide an integer value.")
        return

    SIMULATION_TIME = int(SIMULATION_TIME)

    env = simpy.Environment()
    berth = simpy.Resource(env, capacity=2)
    crane = simpy.Resource(env, capacity=2)
    trucks = [simpy.Resource(env, capacity=1) for _ in range(NUM_TRUCKS)]

    env.process(vessel_generator(env, berth, crane, trucks))
    env.run(until=SIMULATION_TIME)


In [8]:
if __name__ == "__main__":
    main()

Provide the simulation time in minutes: 1000
0: Unloading started for Vessel-1

0: Vessel-1 berthed
3: 1 container unloaded from Vessel-1
3: Container assigned to truck
9: Truck returned
12: 2 container unloaded from Vessel-1
12: Container assigned to truck
18: Truck returned
21: 3 container unloaded from Vessel-1
21: Container assigned to truck
27: Truck returned
30: 4 container unloaded from Vessel-1
30: Container assigned to truck
36: Truck returned
39: 5 container unloaded from Vessel-1
39: Container assigned to truck
45: Truck returned
48: 6 container unloaded from Vessel-1
48: Container assigned to truck
54: Truck returned
57: 7 container unloaded from Vessel-1
57: Container assigned to truck
63: Truck returned
66: 8 container unloaded from Vessel-1
66: Container assigned to truck
72: Truck returned
75: 9 container unloaded from Vessel-1
75: Container assigned to truck
80.69741454898559: Unloading started for Vessel-2

80.69741454898559: Vessel-2 berthed
81: Truck returned
83.697